In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastai.vision.all import *
from fastbook import *

# MNIST Classifier Using PyTorch and FastAI

In [3]:
path = untar_data(URLs.MNIST)

In [4]:
Path.BASE_PATH = path

## Create Train/Validation Sets

In [5]:
training_images = []
validation_images = []
for i in range(0,10):
    training_images.append((path/f'training/{i}').ls().sorted())
    validation_images.append((path/f'testing/{i}').ls().sorted())

In [6]:
training_tensors = []
for images in training_images:
    training_tensors.append(torch.stack([tensor(Image.open(o)) for o in images]).float()/255)

In [7]:
validation_tensors = []
for images in validation_images:
    validation_tensors.append(torch.stack([tensor(Image.open(o)) for o in images]).float()/255)

## Use One-Hot Encoding for the Labels

In [8]:
import torch.nn.functional as F

label_encodings = F.one_hot(torch.arange(0,10)).float()
label_encodings[0]

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
train_x = torch.cat(training_tensors).view(-1,28*28)
val_x = torch.cat(validation_tensors).view(-1,28*28)

train_y = torch.cat([torch.stack([label_encodings[i] for _ in range(len(images))]) 
                 for i, images in enumerate(training_images)])
val_y = torch.cat([torch.stack([label_encodings[i] for _ in range(len(images))]) 
                 for i, images in enumerate(validation_images)])

In [10]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape

(torch.Size([60000, 784]),
 torch.Size([60000, 10]),
 torch.Size([10000, 784]),
 torch.Size([10000, 10]))

In [11]:
training_set = list(zip(train_x, train_y))
validation_set = list(zip(val_x, val_y))

# Create a Model using Softmax and Cross Entropy Loss

In [12]:
# Create a data loader with batch size of 256
dl = DataLoader(training_set, batch_size=256)
valid_dl = DataLoader(validation_set, batch_size=256)
dls = DataLoaders(dl, valid_dl)

In [13]:
def batch_accuracy(xb, yb):
    preds = xb.softmax(1)
    correct = (preds>0.5) == yb
    return correct.float().mean()

def mnist_loss(predictions, targets):
    predictions = predictions.softmax(1)
    return F.cross_entropy(predictions, targets)

simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30,10),
)


In [14]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [15]:
learn.fit(40, .01)

epoch,train_loss,valid_loss,batch_accuracy,time
0,2.306879,2.298369,0.900000,00:04
1,2.303779,2.292974,0.900000,00:01
2,2.300215,2.284804,0.900000,00:01
3,2.296540,2.271742,0.900000,00:01
4,2.292424,2.250248,0.900810,00:01
5,2.284485,2.222245,0.906160,00:01
6,2.270745,2.192522,0.909720,00:01
7,2.254890,2.158380,0.910020,00:01
8,2.237795,2.119126,0.918040,00:01
9,2.219953,2.083408,0.919810,00:01


In [16]:
def predict(o):
    x = (tensor(Image.open(o)).float()/255).view(-1,28*28)
    pred = torch.round(simple_net(x).softmax(1)*100, decimals=0)
    max_cert = 0
    dig = None
    for i in range(0,10):
        certainty = pred[0][i]
        if certainty > max_cert:
            max_cert = certainty
            dig = i
        
    print(f'The digit is a {dig}')
    

In [17]:
Image.open(validation_images[2][100])

In [18]:
predict(validation_images[2][9])

The digit is a 2
